In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly

from timeit import default_timer as timer

# This attempts to re-compile the library in case it has been changed, mostly for debug, but won't do anything
# if nothing is changed
import build_gropt
build_gropt.build_gropt()
import gropt

%matplotlib inline

Building GrOpt . . .


In [2]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

In [3]:
params = {}
params['mode'] = 'free'
params['gmax']  = 0.05
params['smax']  = 50.0
params['moment_params']  = [[0, 0, 0, -1, -1, 11.74, 1.0e-3]]
params['moment_params'].append([1, 0, 0, -1, -1, -11.74, 1.0e-3])
params['moment_params'].append([2, 0, 0, -1, -1, 11.74, 1.0e-3])
params['TE']  = 1.0
params['dt']  = 20e-6
params['Naxis'] = 3

G, dd = gropt.gropt(params, verbose=1)

# print(dd)

In [5]:
from helper_utils import *

eddy_range = [1e-3,120,1000]
diffmode = 0
if params['mode'][:4] == 'diff':
    diffmode = 1
Naxis = params.get('Naxis', 1)
TE = params['TE']
T_readout = params['T_readout']
tINV = TE/2.0

dt = (TE-T_readout) * 1.0e-3 / G.shape[1]
tt = np.arange(G.shape[1]) * dt * 1e3

# trace_list = []
# trace_list.append([go.Scatter(y=G[x]*1000, x=tt) for x in range(3)])
fig = go.Figure()
for i in range(Naxis):
    fig.add_trace(go.Scatter(x=tt, y=G[i]*1000))

for i in range(Naxis):
    fig.add_trace(go.Scatter(x=tt[:-1], y=np.diff(G[i])/dt, visible=False))

    mm = get_moment_plots(G, T_readout, dt, diffmode)
#axarr[i_row, i_col].axhline(linestyle='--', color='0.7')
for i in range(3):
    if diffmode == 1:
        mmt = mm[i]/np.abs(mm[i]).max()
    if diffmode == 0:    
        if i == 0:
            mmt = mm[i]*1e6
        if i == 1:
            mmt = mm[i]*1e9
        if i == 2:
            mmt = mm[i]*1e12
    fig.add_trace(go.Scatter(x=tt, y=mmt, visible=False))
    
eddy_range = [1e-3,120,1000]
all_lam = np.linspace(eddy_range[0],eddy_range[1],eddy_range[2])
all_e = []
for lam in all_lam:
    lam = lam * 1.0e-3
    r = np.diff(np.exp(-np.arange(G[0].size+1)*dt/lam))[::-1]  # TODO: 3-axis case, right now just assumes 1 axis
    all_e.append(100*r@G[0])
fig.add_trace(go.Scatter(x=all_lam, y=all_e, visible=False))

eddy_lines = []
eddy_draw = []
for e in eddy_lines:
    eddy_draw.append(dict(type="line",
        x0=e, x1=e, y0=-1, y1=0,
        line=dict(color="#D64B4B",
            width=1,
            dash="dot")))
eddy_draw.append({"type":"line", "x0":0, "x1":120, "y0":0, "y1":0, "line":{"color":"#777777", "width":1, "dash":"dash"}})
    
pns_lines = [{"type":"line", "x0":0, "x1":1, "y0":1, "y1":1, "line":{"color":"#D64B4B", "width":1, "dash":"dot"}}]
pns = np.abs(get_stim(G, dt))
fig.add_trace(go.Scatter(x=tt[:-1], y=pns, visible=False))

fig.update_layout(width=800,
    height=500,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white",
    updatemenus=[
    dict(active = 0,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.1,
        xanchor="left",
        y=1.1,
        yanchor="top",
        buttons=[
            dict(label = "Gradient",
                 method = "update",
                 args = [{"visible": [True if x < 3 else False for x in range(11)],
                         "shapes":[]}]),
            dict(label = "Slew",
                 method = "update",
                 args = [{"visible": [True if x >= 3 and x < 6 else False for x in range(11)],
                         "shapes":[]}]),
            dict(label = "Moments",
                 method = "update",
                 args = [{"visible": [True if x >= 6 and x < 9 else False for x in range(11)],
                         "shapes":[]}]),
            dict(label = "Eddy",
                 method = "update",
                 args = [{"visible": [True if x == 9 else False for x in range(11)],
                         "shapes":eddy_draw}]),
            dict(label = "PNS",
                 method = "update",
                 args = [{"visible": [True if x > 9 else False for x in range(11)],
                         "shapes":pns_lines}])
            ])])